# Training: SQL (Heavy-users) 💪
Welcome to the training notebook on using SQL.


This notebook is pitched at heavy users who perform data management rples to store objects permanently in SQL.

![SQL Futurama meme](https://live.staticflickr.com/65535/49188193126_3b991de60e_z.jpg "SQL Futurama meme") 

# What will this session cover? 👂
This session will show you how to do the following things in SQL:
1. Dynmaic SQL querying
1. Informal guide to when you should permanently store SQL objects
1. Creating and updating tables
1. Importing data into SQL
1. Indexing columns to improve querying speeds
1. Adding constraints to columns to restrict entries that can go inside it
1. Using stored procedures and functions to do more bespoke operations
1. Database triggers
1. Version-controlling databases

In [4]:
-- Set database to use
USE [HEFE-AN-DEV]

# 1. Dynamic SQL querying
This is essentially SQL code that writes SQL code. It's particularly useful when you want to parameterise your SQL statements with an input variable. 

To be able to employ dynamic SQL querying, you'll need to know how to:
1. Create a variable in SQL and assign it a value
1. Create a SQL query which takes the variable you created
1. Execute the SQL query

The core idea behind dynamic SQL querying lies in being able to write your query as a string/varchar/text, and then execute/run the text as if it is a SQL query.

> **USER STORY:** *As a lazy-ass, punk-ass, funk-ass SQL version of NAS, I want to quickly run a SQL statement to quickly count all the number of rows in several tables in my database rather than write the same* `SELECT COUNT(*) FROM <*table_name*>` *statement several times but with different table names, so I can get back to making my new coding mixtape.*



In [9]:
-- create and set variable/parameters
DECLARE @name_schema AS NVARCHAR(25) = 'Production'
DECLARE @name_table AS NVARCHAR(50) = 'Product'

-- write dynamic SQL as string/text
DECLARE @query AS NVARCHAR(MAX) = 
'
    SELECT COUNT(*)
    FROM [' + @name_schema + '].[' + @name_table + '];
' 

-- print query to see what SQL is written
PRINT @query

-- excute dynamic SQL being written as string/text
EXEC sp_executesql @query

## EXERCISE: Dynamic SQL querying #1
**Question:** Is there an even more efficient way to perform the same task in (1.) without having to use dynamic SQL querying?

**Hint:** Consider using System Tables belonging to the `[INFORMATION_SCHEMA]`.

**Note:** This exercise introduces you to some very useful tables that exist in the background which will be extremely useful for data management tasks.

In [0]:
-- please write your answer here

## EXERCISE: Dynamic SQL querying #2
**Question:** Recall in the *Training: SQL (Medium-user)* notebook the instructions for how you pivot tables in SQL. Namely, how you have to explicitly know and code the rows in the column you want to pivot to. This can be tedious, especially when you have many unique rows to pivot to. There is a way to pivot without explicitly specifying the rows you want to pivot to columns. Can you write a query that does this?

**Hint:** This exercise demonstrates the power of dynamic querying.

# 2. Informal guide to what you should permanently store in SQL